## HW4 Part A: Logistic Regression for Digits Data Set

Blanca Miller
<br>
CS 791
<br>
03/02/2018

__Objective:__ Generate a logistic regression model to model the posterior probabilities for the K classes using linear functions in X. We estimate the probability that the outcome variable assumes a certain value, rather than estimating the value itself. 